In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stocks/gr500182.csv
/kaggle/input/stocks/gr500209.csv
/kaggle/input/stocks/gr500680.csv
/kaggle/input/stocks/gr530965.csv
/kaggle/input/stocks/gr532174.csv
/kaggle/input/stocks/gr532210.csv
/kaggle/input/stocks/gr532540.csv
/kaggle/input/stocks/gr500325.csv
/kaggle/input/stocks/gr500112.csv
/kaggle/input/stocks/gr500180.csv
/kaggle/input/stocks/gr507685.csv


In [2]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from math import sqrt
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback

import prettytable

In [3]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1))

    return (x_train, x_test, y_train, y_test)

In [ ]:
def create_confusion_matrix(y_pred,y_true):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [ ]:
def create_model(x_train,y_train, epochs = 50, batch_size = 32):
    
    model = Sequential()
    model.add(Conv1D(32, 1, activation="relu", input_shape=(x_train.shape[1],1)))
    model.add(Flatten())
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1,activation='softmax'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='sgd')
    model.fit(x_train, y_train, batch_size=batch_size,epochs=epochs,verbose=0)
    return model

In [ ]:
def cnn_classification(df,column = "Next Day Close Price GR",epochs = 50,batch_size = 32,threshold = 0.01):
    df["Target"] = df[column].apply(lambda x : 1 if x >= threshold else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    (x_train, x_test, y_train, y_test) = reshape_data(x_train, x_test, y_train, y_test)
    
    model = create_model(x_train,y_train,epochs = 50,batch_size = 32)
    y_pred = model.predict(x_test)
    myresult = create_confusion_matrix(y_pred,y_test)
    myresult.update({"epochs":epochs,"batch_size":batch_size,"threshold":threshold})
    print("done")
    return myresult


In [ ]:
combinations = list()
def get_combinations(arr):
    n = len(arr)
    indices = [0 for i in range(n)]
    while (1):
        res = list()
        for i in range(n):
            res.append(arr[i][indices[i]])
        combinations.append(res)
        next = n - 1
        while (next >= 0 and
              (indices[next] + 1 >= len(arr[next]))):
            next-=1
        if (next < 0):
            return
        indices[next] += 1
        for i in range(next + 1, n):
            indices[i] = 0
b = [25,32,64,100]
e = [25,50,100]
t = [0.01,0.02,0.03,0.04,0.05]

arr = [b,e,t]
get_combinations(arr)

In [ ]:
%%time
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(filename)
        if os.path.exists(str(filename[2:8])+"_cnn.csv"):
            continue
        filepath = os.path.join(dirname, filename)
        df = pd.read_csv(filepath)
        column = "Next Day Close Price GR"
        df = pre_process_data(df,60)
        (df,column) = dependent_column(df,column)
        arguments = list()
        for batch_size,epochs,threshold in combinations:
            data = [df,column,epochs,batch_size,threshold]
            arguments.append(data)
        threads = ThreadPool(4)
        result = threads.starmap(cnn_classification,arguments[:])
        resultdf = pd.DataFrame(result)
        resultdf.to_csv(os.path.join(os.getcwd(),str(filename[2:8])+"_cnn"+".csv"),index=None)

In [98]:
best_cnn = pd.DataFrame()
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        
        if filename.endswith(".csv") and "best" not in filename:
            filepath = os.path.join(dirname, filename)
            df = pd.read_csv(filepath)
            table = prettytable.PrettyTable()
            table.title = str(filename[:6]) + " cnn "
            fields = list(df.columns)
            fields = fields[-3:]+fields[:-4]
            df = df[fields]
#             fields.remove("confusion matrix")
            table.field_names = fields
            for _, row in df.iterrows():
                row = [round(r,6) for r in row if isinstance(r,(float,int))]
                table.add_row(row)
            print(table)
            df["security id"] = filename[:6]
            for name, log in df.groupby(by=["threshold"]):
                log = log.sort_values(by=["f1_score"],ascending=[False])
                cols = log.columns.tolist()
                log = log[[cols[-1]]+cols[:-1]]
                best_cnn = best_cnn.append(log.head(1),ignore_index=True)
best_cnn.to_csv("best_cnn.csv",index=None)

+----------------------------------------------------------------------------+
|                                500325 cnn                                  |
+--------+------------+-----------+----------+-----------+--------+----------+
| epochs | batch_size | threshold | accuracy | precision | recall | f1_score |
+--------+------------+-----------+----------+-----------+--------+----------+
|  25.0  |    25.0    |    0.01   | 0.050056 |  0.050056 |  1.0   | 0.095339 |
|  25.0  |    25.0    |    0.02   | 0.101224 |  0.101224 |  1.0   | 0.183838 |
|  25.0  |    25.0    |    0.03   | 0.050056 |  0.050056 |  1.0   | 0.095339 |
|  25.0  |    25.0    |    0.04   | 0.026696 |  0.026696 |  1.0   | 0.052004 |
|  25.0  |    25.0    |    0.05   | 0.050056 |  0.050056 |  1.0   | 0.095339 |
|  50.0  |    25.0    |    0.01   | 0.252503 |  0.252503 |  1.0   | 0.403197 |
|  50.0  |    25.0    |    0.02   | 0.101224 |  0.101224 |  1.0   | 0.183838 |
|  50.0  |    25.0    |    0.03   | 0.050056 |  0.05

+----------------------------------------------------------------------------+
|                                532174 cnn                                  |
+--------+------------+-----------+----------+-----------+--------+----------+
| epochs | batch_size | threshold | accuracy | precision | recall | f1_score |
+--------+------------+-----------+----------+-----------+--------+----------+
|  25.0  |    25.0    |    0.01   | 0.053452 |  0.053452 |  1.0   | 0.10148  |
|  25.0  |    25.0    |    0.02   | 0.163697 |  0.163697 |  1.0   | 0.28134  |
|  25.0  |    25.0    |    0.03   | 0.092428 |  0.092428 |  1.0   | 0.169215 |
|  25.0  |    25.0    |    0.04   | 0.053452 |  0.053452 |  1.0   | 0.10148  |
|  25.0  |    25.0    |    0.05   | 0.053452 |  0.053452 |  1.0   | 0.10148  |
|  50.0  |    25.0    |    0.01   | 0.291759 |  0.291759 |  1.0   | 0.451724 |
|  50.0  |    25.0    |    0.02   | 0.163697 |  0.163697 |  1.0   | 0.28134  |
|  50.0  |    25.0    |    0.03   | 0.092428 |  0.09

In [99]:
besttable = prettytable.PrettyTable()
besttable.title = "best cnn"
besttable.field_names = best_cnn.columns
for _, row in best_cnn.iterrows():
    row = [round(r,6) if isinstance(r,(float,int)) else r for r in row]
    besttable.add_row(row)
print(besttable)

+------------------------------------------------------------------------------------------+
|                                         best cnn                                         |
+-------------+--------+------------+-----------+----------+-----------+--------+----------+
| security id | epochs | batch_size | threshold | accuracy | precision | recall | f1_score |
+-------------+--------+------------+-----------+----------+-----------+--------+----------+
|    500325   |   50   |     25     |    0.01   | 0.252503 |  0.252503 |  1.0   | 0.403197 |
|    500325   |   25   |     25     |    0.02   | 0.101224 |  0.101224 |  1.0   | 0.183838 |
|    500325   |   25   |     64     |    0.03   | 0.101224 |  0.101224 |  1.0   | 0.183838 |
|    500325   |   50   |     25     |    0.04   | 0.050056 |  0.050056 |  1.0   | 0.095339 |
|    500325   |   25   |     25     |    0.05   | 0.050056 |  0.050056 |  1.0   | 0.095339 |
|    507685   |   50   |     25     |    0.01   | 0.215402 |  0.215402